In [4]:
from glob import glob
from tqdm import tqdm
import pandas as pd
import requests
import json

headers = {
    'accept': 'application/json',
    'Authorization': 'Bearer',
    'Content-Type': 'application/json',
}

json_data = {
    'input': 'a',
    'to_lang': 'ms',
    'model': 'small',
    'temperature': 0,
    'top_p': 1,
    'top_k': 1,
    'repetition_penalty': 1.1,
}

response = requests.post('https://api.mesolitica.com/translation', headers=headers, json=json_data)

In [6]:
response.json()['result']

'a'

In [2]:
files = glob('generate-persona/*.json')
files.extend(glob('generate-persona-wikipedia/*.json'))
len(files)

176493

In [22]:
data = []
for f in tqdm(files):
    with open(f) as fopen:
        d = json.load(fopen)
        d['persona'] = d.pop('r')
        data.append(d)

100%|████████████████████████████████████████████████████████████████████████████| 176493/176493 [00:04<00:00, 42780.47it/s]


In [23]:
with open('persona.json', 'w') as fopen:
    json.dump(data, fopen)

In [25]:
!mkdir translate-persona

In [26]:
(len(data) // 2) * 0

0

In [27]:
(len(data) // 2) * 1

88246

In [32]:
import os
import time

for i in tqdm(range((len(data) // 3) * 0, (len(data) // 3) * 1, 1)):
    filename = os.path.join('translate-persona', f'{i}.json')
    if os.path.exists(filename) and os.path.getsize(filename) > 100:
        continue
    d = data[i]
    json_data = {
        'input': d['persona'],
        'to_lang': 'ms',
        'model': 'small',
        'temperature': 0,
        'top_p': 1,
        'top_k': 1,
        'repetition_penalty': 1.1,
    }
    
    while True:
        try:

            response = requests.post('https://api.mesolitica.com/translation', headers=headers, json=json_data)
            r = response.json()['result']
            with open(filename, 'w') as fopen:
                json.dump(r, fopen)
            break
        except:
            time.sleep(60)

100%|███████████████████████████████████████████████████████████████████████████████| 58831/58831 [9:55:07<00:00,  1.65it/s]


In [40]:
filtered = []
for i, d in tqdm(enumerate(data)):
    filename = filename = os.path.join('translate-persona', f'{i}.json')
    if not os.path.exists(filename):
        continue
    
    with open(filename) as fopen:
        r = json.load(fopen)
    
    s = r.split()
    if (len(set(s)) / len(s)) > 0.5:
        d['persona_ms'] = r
        filtered.append(d)

176493it [00:24, 7138.27it/s]


In [41]:
df = pd.DataFrame(filtered)
df.to_parquet('MY-persona.parquet')

In [42]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="MY-persona.parquet",
    path_in_repo='data/persona-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-Persona",
    repo_type="dataset",
)

MY-persona.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Persona/commit/19826ddc01ce0ae26922b4ce1c24abfa09bc8765', commit_message='Upload data/persona-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='19826ddc01ce0ae26922b4ce1c24abfa09bc8765', pr_url=None, pr_revision=None, pr_num=None)